# Plot Tracts and Patches

 work with Weakly_2024_10
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/04/10
- update : 2024/04/10


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_10/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+399544b0f1 	current w_2024_10 setup


In [3]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize


from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

# LSST packages
from lsst.daf.butler import Butler
import lsst.geom as geom

# allow interactive plots
#%matplotlib widget
%matplotlib inline

# default plot style is accessible
plt.style.use('tableau-colorblind10')

In [4]:
# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collectionn = 'LSSTComCamSim/runs/nightlyvalidation/2024040X/d_2024_03_29/DM-43612'
#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"

In [5]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collections, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
print("camera_name = {}".format(camera.getName()))
print("collections = {}".format(collections))

camera_name = LSSTComCamSim
collections = ['LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612']


### access to skymap

https://github.com/sylvielsstfr/MyDP0LSS/blob/main/myDP0.2/TrackAndPatch/PlotTractAndPatchs_butler.ipynb

In [6]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

## Config

In [7]:
datasetRefs = butler.registry.queryDatasets(datasetType='skyMap', collections=collections, where= where_clause)

In [8]:
for i, ref in enumerate(datasetRefs):
    print(ref)
    skymap_id = ref.dataId["skymap"]
    if skymap_id ==  skymapName:
        skymap = ref

skyMap@{skymap: 'hsc_rings_v1'} [sc=SkyMap] (run=skymaps id=6e7fc928-0dc3-4e1b-ae63-5fb63be46216)
skyMap@{skymap: 'latiss_v1'} [sc=SkyMap] (run=skymaps id=f677ff86-2b1c-4094-9dad-e39395052cd1)
skyMap@{skymap: 'lsst_cells_v1'} [sc=SkyMap] (run=skymaps id=857e0e80-8d08-44f4-a81b-84227b064d5b)
skyMap@{skymap: 'DC2_cells_v1'} [sc=SkyMap] (run=skymaps id=b7d4b4f1-e108-4570-bbf5-da6093179ac8)
skyMap@{skymap: 'ops_rehersal_prep_2k_v1'} [sc=SkyMap] (run=skymaps id=8f5f7e7a-c647-4a88-b8e7-6e2406d071b8)


In [9]:
nImage_refs = list(butler.registry.queryDatasets('deepCoadd_nImage'))
my_tracts = np.unique([ref.dataId['tract'] for ref in nImage_refs])
my_tracts = sorted(my_tracts)
print(my_tracts)

[2494, 2495, 2661, 2662, 3200, 3345, 3346, 3384, 3385, 3533, 3534, 6914, 6915, 7148, 7149, 7445, 7446, 7683, 7684, 9570, 9637, 9638, 9812, 9813, 9880, 9881]


In [10]:
#from lsst.daf.butler import Butler
#butler = Butler('SMALL_HSC')
#skymap = butler.get(ref)
skymap = butler.get('skyMap', skymap=skymapName, collections=collections)
tractInfo = skymap.generateTract(my_tracts[0])
patch = tractInfo[41]
patch.getIndex()

Index2D(x=7, y=2)

In [11]:
# Load data (deserialize)
import pickle
#with open('Latiss_tractspatchs.pickle', 'rb') as handle:
with open('../data/202402/Latiss_tractspatchs_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../data/202402/Latiss_tractspatchs_202402.pickle'

In [ ]:
tracts_patchs_dict

In [ ]:
list_of_tracts = list(tracts_patchs_dict.keys())
list_of_tracts

In [ ]:
npatches = 0
for itract in list_of_tracts:
    npatches += len(tracts_patchs_dict[itract])
print(npatches)    

In [ ]:
#corners = np.array([ (ll[0], ll[1]), 
#                    (ul[0], ul[1] ), 
#                    (ur[0], ur[1]), 
#                    (lr[0], lr[1])])

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

numTracts=0
# collection of polygons
patches = []
all_ra = []
all_dec = []
for tractInfo in skymap:
    tractID = tractInfo.getId()
    if tractID not in list_of_tracts:
        continue
    # Work with the new tract    
    numTracts+=1
    all_xvals = []
    all_yvals = []
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID not in tracts_patchs_dict[tractID]:
            continue
        ibb=patch.getInnerBBox()
        tWCS=tractInfo.getWcs()
        corners = []
        # loop on the 4 corners
        for icorn,corner in enumerate(ibb.getCorners()):
            p = geom.Point2D(corner.getX(), corner.getY())
            coord = tWCS.pixelToSky(p)
            corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
            all_ra.append(coord.getRa().asDegrees()) 
            all_dec.append(coord.getDec().asDegrees()) 
        print(f"==={numTracts}===tractId:patchId = {tractID}:{patchID}==========")    
        corners_array = np.array(corners)
        corners_array = np.append(corners_array,[corners_array[0]],axis=0)
        print("corners = ",corners_array)
        polygon = Polygon(corners_array,closed=True)
        patches.append(polygon)
print(numTracts)   

# we can see this, not my polugons ???
#num_polygons = 5
#num_sides = 4
#for i in range(num_polygons):
#    polygon = Polygon(10*(np.random.rand(num_sides ,2)-0.5), True)
#    patches.append(polygon)



fig, ax = plt.subplots(figsize=(12,12))
p = PatchCollection(patches, cmap=mpl.cm.jet, alpha=0.4)

colors = 100*np.random.rand(len(patches))
p.set_array(np.array(colors))
#p.set_clim([5, 50])
ax.add_collection(p)  
fig.colorbar(p)

#ax.set_xlim([0., 60.])
#ax.set_ylim([-90, -30.])
fig.show()

In [ ]:
all_ra = np.array(all_ra)
all_dec = np.array(all_dec)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
p = PatchCollection(patches, cmap=mpl.cm.jet, alpha=0.4)

colors = 50*np.random.rand(len(patches))
p.set_array(np.array(colors))
p.set_clim([5, 50])
ax.add_collection(p)  


#ax.set_xlim([0., 360.])
#ax.set_ylim([-90, 10.])
plt.show()

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

fig, ax = plt.subplots()
patches = []
num_polygons = 5
num_sides = 4

for i in range(num_polygons):
    polygon = Polygon(10*np.random.rand(num_sides ,2), closed=True)
    patches.append(polygon)

p = PatchCollection(patches, cmap=matplotlib.cm.jet, alpha=0.4)

colors = 100*np.random.rand(len(patches))
p.set_array(np.array(colors))

ax.add_collection(p)

ax.set_xlim([0., 10.])
ax.set_ylim([0., 10.])

plt.show()